In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import tensorflow as tf
import os
import zipfile
local_zip = '/content/drive/My Drive/car&bike.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content/car&bike')
zip_ref.close()



    

In [0]:
train_car_dir = os.path.join('/content/car&bike/car&bike/car')

train_bike_dir= os.path.join('/content/car&bike/car&bike/bike')

print('total training car images:', len(os.listdir(train_car_dir)))
print('total training bike images:', len(os.listdir(train_bike_dir)))


    

total training car images: 696
total training bike images: 699


In [0]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255,
                                  rotation_range=40,
                                  width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Flow training images in batches of 128 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        '/content/car&bike/car&bike',  # This is the source directory for training images
        target_size=(300, 300),  # All images will be resized to 150x150
        batch_size=128,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')


Found 1395 images belonging to 2 classes.


In [0]:
model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 300x300 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fifth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    # Only 1 output neuron. It will contain a value from 0-1 where 0 for 1 class ('car') and 1 for the other ('bike')
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 35, 35, 64)        0         
____

In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(lr=0.001),
              metrics=['acc'])

In [0]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=10,  
      epochs=15,
      verbose=1)


Epoch 1/15
10/10 [==============================] - 33s 3s/step - loss: 1.0364 - acc: 0.5335
Epoch 2/15
10/10 [==============================] - 32s 3s/step - loss: 0.8407 - acc: 0.6038
Epoch 3/15
10/10 [==============================] - 31s 3s/step - loss: 0.6445 - acc: 0.8208
Epoch 4/15
10/10 [==============================] - 32s 3s/step - loss: 0.3585 - acc: 0.8366
Epoch 5/15
10/10 [==============================] - 32s 3s/step - loss: 0.4117 - acc: 0.8485
Epoch 6/15
10/10 [==============================] - 32s 3s/step - loss: 0.3166 - acc: 0.8594
Epoch 7/15
10/10 [==============================] - 32s 3s/step - loss: 0.2959 - acc: 0.8856
Epoch 8/15
10/10 [==============================] - 32s 3s/step - loss: 0.3125 - acc: 0.8732
Epoch 9/15
10/10 [==============================] - 33s 3s/step - loss: 0.2772 - acc: 0.8797
Epoch 10/15
10/10 [==============================] - 31s 3s/step - loss: 0.2085 - acc: 0.9219
Epoch 11/15
10/10 [==============================] - 33s 3s/step - lo

In [0]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(300, 300))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a car")
  else:
    print(fn + " is a bike")
 

Saving 0314.jpg to 0314.jpg
Saving 0315.jpg to 0315.jpg
Saving 0316.jpg to 0316.jpg
Saving 0320.jpg to 0320.jpg
Saving 0321.jpg to 0321.jpg
Saving 0322.jpg to 0322.jpg
[0.]
0314.jpg is a bike
[0.]
0315.jpg is a bike
[0.]
0316.jpg is a bike
[0.]
0320.jpg is a bike
[0.]
0321.jpg is a bike
[0.]
0322.jpg is a bike
